In [3]:
import os

from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

load_dotenv(dotenv_path="../.env", override=True)

True

In [4]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question．
ユーザーの質問に基づいてベクトルストアから取得されたドキュメントを返します。
"""

@traceable(run_type="chain")
def retrieve_documents(question:str)->str:
    return retriever.invoke(question)

## Simple RAG application
このノートブックでは、LangSmithについて学びながら使用するシンプルなRAGアプリケーションを設定します。

RAG (Retrieval Augmented Generation) とは、LLM（大規模言語モデル）に関連するドキュメントを提供して、ユーザーの質問により適切に答えられるようにするための一般的な手法です。

今回は、LangSmithのドキュメントをインデックス化してみます！

LangSmith は、LangChainを必要とせず、どんなLLMアプリケーションでも簡単に追跡できるようにするツールです。


In [6]:
"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question．
ユーザーの質問に基づいてベクトルストアから取得されたドキュメントを返します。
"""

@traceable(run_type="chain")
def retrieve_documents(question:str)->str:
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question:str, documents:str)->str:
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"以下の文脈をもとに日本語で返答してください。\n\n文脈: {formatted_docs}\n\n 質問: {question}"
        },   
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(run_type="llm")
def call_openai(
    messages: List[dict],model:str=MODEL_NAME,temperature:float=0.0
)->str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
        ) 

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content



これには1分弱かかる予定です。LangSmithのドキュメントをインデックス化し、SKLearnベクトルデータベースに保存しています

In [7]:
question = "What is LangSmith used for?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"website": "www.google.com"}})
print(ai_answer)

LangSmithは、LLMアプリケーションの開発、監視、テストのためのプラットフォームです。プロトタイピングやデバッグ、パフォーマンス評価を行い、アプリケーションの信頼性を高めることができます。また、テスト駆動型のアプローチを採用し、データセットを作成してアプリケーションのテストを実行することも可能です。
